In [2]:
import pybullet
import gym
import numpy as np
import matplotlib.pyplot as plt
import pybullet_data
import math
import rrtstar
import mpc
import time
import torch
from environment import Environment
from maddpg import MADDPGAgentTrainer
import tensorflow as tf
import tensorflow.keras.layers as layers

In [ ]:
#!git clone https://github.com/ROBOTIS-GIT/turtlebot3.git

In [ ]:
# RRT
goal = (5.25, 3.5, 0)
start = (0.25, 0.25, 0)
samplingArea = [0, 10, 0, 5]
RRT = rrtstar.RRT(start, goal, [], samplingArea, 'rrtstar', 'rectangle', 2)
path = RRT.planning()

In [22]:
print(path)
path = np.flip(path, 0)
print(path)


[[5.25, 3.5, 0], [5.25, 3.5, 0], [0.25, 0.25, 0]]
[[0.25 0.25 0.  ]
 [5.25 3.5  0.  ]
 [5.25 3.5  0.  ]]


In [32]:
extended_path = []
for i in range(len(path)):
    print(path[i])
for i in range(len(path)):
    if i + 1 != len(path) - 1:
        angle = 0.39
        path_part = list(zip(np.linspace(path[i][0], path[i+1][0], 10), np.linspace(path[i][1], path[i+1][1], 100)))
        for part in path_part:
            extended_path.append((part[0], part[1], angle))
    else:
        break
        
print(extended_path)

[0.25 0.25 0.  ]
[5.25 3.5  0.  ]
[5.25 3.5  0.  ]
[(0.25, 0.25, 0.39), (0.8055555555555556, 0.6111111111111112, 0.39), (1.3611111111111112, 0.9722222222222222, 0.39), (1.9166666666666667, 1.3333333333333333, 0.39), (2.4722222222222223, 1.6944444444444444, 0.39), (3.0277777777777777, 2.0555555555555554, 0.39), (3.5833333333333335, 2.4166666666666665, 0.39), (4.138888888888889, 2.7777777777777777, 0.39), (4.694444444444445, 3.138888888888889, 0.39), (5.25, 3.5, 0.39)]


In [24]:
physicsClient = pybullet.connect(pybullet.GUI)
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())
pybullet.setGravity(0,0,-9.8)

# Loading chessgrid plane
planeID = pybullet.loadURDF('plane.urdf')
chess_grid = pybullet.loadTexture('checker_huge.gif')
pybullet.changeVisualShape(planeID, -1, textureUniqueId=chess_grid)

# Setting up overhead camera
viewMatrix = pybullet.computeViewMatrix(
    cameraEyePosition=[0.75,0.25,3],
    cameraTargetPosition=[0.75,0.25,0],
    cameraUpVector=[0,1,0])

projectionMatrix = pybullet.computeProjectionMatrixFOV(
    fov=45.0,
    aspect=1.0,
    nearVal=0.1,
    farVal=3.1)

width, height, rgbImg, depthImg, segImg = pybullet.getCameraImage(width = 224, height = 224, viewMatrix=viewMatrix, projectionMatrix = projectionMatrix)
player = pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', start, pybullet.getQuaternionFromEuler([0,0,0]))
pybullet.changeVisualShape(player, 0, rgbaColor=[0,0,1,1])

pybullet.setRealTimeSimulation(1)

In [13]:
def applyAction(actions, orig_state):
    new_states = []
    for i in range(G):
        action_set = actions[i]
        for j in range(H):
            action = action_set[j]
            action_3d = [fM * action[0], fM * action[1], 0]
            for x in range(10):
                pybullet.stepSimulation()
                time.sleep(1./240.)
                current_pos = pybullet.getBasePositionAndOrientation(player)[0]
                #pybullet.applyExternalForce(player, -1, action_3d, current_pos, pybullet.WORLD_FRAME)
                pybullet.resetBaseVelocity(player, action_3d)
            pybullet.resetBaseVelocity(player, [0,0,0])

        
        new_states.append(pybullet.getBasePositionAndOrientation(player))
        pybullet.resetBasePositionAndOrientation(player, orig_state[0], orig_state[1])
    return new_states
        

In [14]:
def getCost(states, target):
    #target = [goal[0], goal[1], 0]
    target_pos = [target[0], target[1]]
    target_angle = target[2]
    costs = []
    for i in range(G):
        current = states[i]
        current_pos = current[0]
        current_angle = pybullet.getEulerFromQuaternion(current[1])[2]
        # Double check above. Based on angle used for initial setting position
        cost = math.sqrt(((current_pos[0] - target_pos[0]) ** 2) + ((current_pos[1] - target_pos[1]) ** 2))
        cost += abs(current_angle - target_angle)
        costs.append(cost)
    return costs

In [28]:
H = 5
G = 100
T = 10
K = 5
fM = 1

def cem(path):
    pybullet.setRealTimeSimulation(0)
    total_actions = []
    for i in range(len(path) - 1):
    #for i in range(1000):
        initial_actions = np.random.normal(0,1,(G,H,2))
        state = pybullet.getBasePositionAndOrientation(player)
        print(state)
        for t in range(T):
            new_states = applyAction(initial_actions, state)
            costs = getCost(new_states, path[i + 1])
            #costs = getCost(new_states, path[1])
            sorted_actions = [act for _,act in sorted(zip(costs, initial_actions), key=lambda pair: pair[0])]
            new_mean = np.mean(sorted_actions[:K])
            new_std = np.std(sorted_actions[:K])
            new_actions = np.random.normal(new_mean, new_std, (G,H,2))
            #print(sorted_actions[:K])
            #print(new_actions)
            initial_actions = np.concatenate((sorted_actions[:K], new_actions))
            
        best_action = initial_actions[0][0] # Because no horizon length
        action = [fM * best_action[0], fM * best_action[1], 0]
        total_actions.append(action)
        #print(action)
        for x in range(10):
            pybullet.stepSimulation()
            time.sleep(1./240.)
            #pybullet.applyExternalForce(player, -1, action, state[0], pybullet.WORLD_FRAME)
            pybullet.resetBaseVelocity(player, action)
        pybullet.resetBaseVelocity(player, [0,0,0])
    return total_actions

            
            


In [29]:
actions = cem(extended_path)
#pybullet.setRealTimeSimulation(1)

((0.25, 0.25, 0.0), (0.0, 0.0, 0.0, 1.0))
((0.2942801704783471, 0.26973208553270533, 0.0005770321779222394), (-0.0004972244114593114, 0.03374208551789397, 0.015751060759447866, 0.9993063236654415))
((0.3540693317430558, 0.3151125635695497, 0.0009368717012661425), (-0.0019120523761195112, 0.06506262004658113, 0.029840277050655447, 0.9974330841685197))
((0.477343914141943, 0.43907374511668057, 0.0025463472275899235), (-0.00434745607234183, 0.09183387669011345, 0.03905561801965592, 0.9949986419181017))
((0.5532986463644156, 0.5276024533861382, 0.0015548535934238055), (-0.0033256706194210997, 0.08378708748124403, 0.03571854942170047, 0.9958377624460477))
((0.6768174795197334, 0.5898201105327558, 0.0014789193127153317), (-0.002748231532751313, 0.08283467680267648, 0.031299789280733584, 0.9960678625141061))
((0.761083064879749, 0.6468506974244999, 0.0013466646711185182), (-0.002279689651870782, 0.08171380108665326, 0.027029693615103118, 0.9962866321446533))
((0.9288445032259143, 0.7447634071

In [30]:
print(actions)

[[1.2010228056113257, 0.5547382488199697, 0], [1.6129398559441022, 1.2367355143073044, 0], [3.3079716820725107, 3.3469373793115396, 0], [2.0273259179163183, 2.3689243243781837, 0], [3.308058439633955, 1.6739150862415417, 0], [2.2574475485382814, 1.5361480603266817, 0], [4.488506251779748, 2.6290656088594475, 0], [2.5241656129606023, 3.0453374087767258, 0], [2.2927930616189043, 2.710495429295285, 0]]


In [33]:
pybullet.resetBasePositionAndOrientation(player, [0.25,0.25,0], pybullet.getQuaternionFromEuler([0,0,0]))
for action in actions:
    for x in range(10):
        pybullet.stepSimulation()
        time.sleep(1./240.)
        #pybullet.applyExternalForce(player, -1, action, state[0], pybullet.WORLD_FRAME)
        pybullet.resetBaseVelocity(player, action)
    pybullet.resetBaseVelocity(player, [0,0,0])

In [20]:
pybullet.disconnect()